In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.42,19.23,51.01,92,0,7.78,ZA,1595100525
1,1,saskylakh,71.92,114.08,44.01,75,21,12.37,RU,1595100411
2,2,thompson,55.74,-97.86,69.80,52,75,8.05,CA,1595100435
3,3,rikitea,-23.12,-134.97,72.75,80,6,24.94,PF,1595100529
4,4,hobart,-42.88,147.33,50.00,71,0,6.93,AU,1595100355


## Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
135,135,porto torres,40.84,8.40,75.20,69,0,9.17,IT,1595101030
162,162,zhanaozen,43.30,52.80,79.70,41,0,9.17,KZ,1595101034
173,173,omboue,-1.57,9.26,72.77,85,0,8.34,GA,1595100464
206,206,kasongo-lunda,-6.47,16.82,72.16,33,0,2.37,CD,1595101043
239,239,birjand,32.87,59.22,77.00,22,0,9.17,IR,1595101049
305,305,tukrah,32.53,20.58,76.82,71,0,9.73,LY,1595100552
349,349,muros,42.78,-9.06,79.00,44,0,1.99,ES,1595101067
352,352,senftenberg,51.52,14.02,73.99,68,0,2.42,DE,1595101067
382,382,altay,47.87,88.12,72.39,36,0,6.22,CN,1595100577
412,412,bojnurd,37.47,57.33,73.40,38,0,2.24,IR,1595101076


## Hotel Map

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
135,porto torres,IT,40.84,8.40,
162,zhanaozen,KZ,43.30,52.80,
173,omboue,GA,-1.57,9.26,
206,kasongo-lunda,CD,-6.47,16.82,
239,birjand,IR,32.87,59.22,
305,tukrah,LY,32.53,20.58,
349,muros,ES,42.78,-9.06,
352,senftenberg,DE,51.52,14.02,
382,altay,CN,47.87,88.12,
412,bojnurd,IR,37.47,57.33,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
135,porto torres,IT,40.84,8.40,Hotel Elisa
162,zhanaozen,KZ,43.30,52.80,K.korabay 28A
173,omboue,GA,-1.57,9.26,Hotel Olako
206,kasongo-lunda,CD,-6.47,16.82,
239,birjand,IR,32.87,59.22,Birjand Tourist Hotel
305,tukrah,LY,32.53,20.58,استراحة لاقروتا
349,muros,ES,42.78,-9.06,Apartment Av. Castelao
352,senftenberg,DE,51.52,14.02,Wellnesshotel Seeschlösschen - Privat-SPA & Na...
382,altay,CN,47.87,88.12,Jinqiao Hotel
412,bojnurd,IR,37.47,57.33,Hotel negin


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))